In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated

llm = init_chat_model("openai:gpt-4o-mini")
llm.invoke([{"role": "user", "content": "Hello, how are you?"}])

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DB4vVxyGPXpazYU1P6avYiLO4Sn2g', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c7797-187e-73a3-9d82-a4a657f698ab-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


graph_builder = StateGraph(State)

In [5]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


In [ ]:
graph = graph_builder.compile()

graph.invoke(
    {
        "messages": [
        {"role": "user", "content": "Hello, how are you?"}
        ]
    }
)

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='feadaae7-d552-442c-9cb7-754d2ee5f7d8'),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DB4vqzuFkkuRlyx5weK3BvF93WfA9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c7797-6b9c-74f0-b899-bc92acefc3fb-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43, 'input_token_details': {'audio': 0, 